In [1]:
#################################
# filenames
#################################

number = 10

probs_file = 'newprobs'+str(number)+'.json'
filename3s = 'newprobs'+str(number)+'genre.json'
filename3se = 'newprobs'+str(number)+'e.txt'
histo_file = 'histo'+str(number)+'.txt'
url_dict_file = 'url.json'


##############################
# setup
##############################

import requests
import time
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import random

tstart = time.time()
print(tstart)

api_url = 'https://api.hooktheory.com/v1/'
login = {"username": "jl8429",
         "password": ""}

r = requests.post(api_url+'users/auth', data=login)
#print('r', r)
#print(r.json())
'''
{'id': 136206, 'username': 'myUsername', 'activkey': 'myKey'}
'''

activkey = '0e11e1d65a021d0a883f309f813d74ad'
header = {"Authorization": "Bearer "+activkey}


#################################
# RELOAD url_dict
#################################

with open(url_dict_file, 'r') as f:
    url_dict = json.load(f)

big_dict = {}


#################################
# continue
#################################

#with open('chordProbs3new.json', "r") as f:
with open(probs_file, "r") as f:
    data3 = json.load(f)
f2 = open(filename3se, 'w')

j = 1
totals = str(len(data3))
print(totals)

histo = [0]*5000

driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")

for d in data3:
    chord1, chord2, chord3, chord4 = d['child_path'].split(",")
    mykey = str(chord1)+','+str(chord2)+','+str(chord3)+','+str(chord4)
    print(str(j)+'/'+totals, mykey)
    song_list = []
    
    # don't overwrite what we've done already
    if not big_dict.get(mykey, False):
        try:
            myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
            
            # slow down...
            time.sleep(1.1)
            if str(myData)=="<Response [429]>":
                #print("too many requests")
                time.sleep(30)
                myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
                if str(myData)=="<Response [429]>":
                    time.sleep(30)
                    print("too many requests")
            
            page = 1
            keepGoing = True
            #while myData.content and myData.content != b'No songs match this chord progressionnull':
            while keepGoing:
                # can't parse if it's this byte string
                if myData.content != b'No songs match this chord progressionnull':
                    #print('page', page)
                    # parse each song
                    try:
                        my_json = myData.json()
                        songs = len(my_json)
                        #print('songs len', songs)
                        if songs:
                            # for each song, pull the genre, add it to the url_dict and big_dict, then write it to the file
                            for i in range(songs):
                                url = my_json[i]['url'].split("#")[0]
                                #print(url)
                                # check if already in url_dict to save us from having to load the page slowly and parse
                                if url in url_dict:
                                    #print(url, url_dict[url])
                                    my_json[i]['genre'] = url_dict[url]
                                # if not in dict, we need to parse page and add to dict
                                else:
                                    driver.get(url)
                                    html = driver.page_source
                                    soup = BeautifulSoup(html, 'html.parser')
                                    try:
                                        #genre = soup.find_all(['multiselect'])[0].find('a').contents[0]
                                        # some songs don't have a genre and some have multiple genres...
                                        genres = soup.find_all(['multiselect'])[0].find_all('a')
                                        gk = 0
                                        glist = []
                                        for g in genres:
                                            # .find_all() will return a lot of garbage, but the genres will have a length of 1
                                            if len(g) > 1:
                                                # if even the first is garbage, it has no genre
                                                if gk == 0:
                                                    my_json[i]['genre'] = ["Unknown"]
                                                    url_dict[url] = ["Unknown"]
                                                break
                                            else:
                                                glist.append(g.contents[0])
                                            gk += 1
                                        my_json[i]['genre'] = glist
                                        url_dict[url] = glist
                                    except:
                                        print('3. Unable to extract genre for', url)
                                        my_json[i]['genre'] = ["Unknown"]
                                        url_dict[url] = ["Unknown"]

                                    # this is not where this block belongs
                                    # nesting it under "else" above made failures generate "[]" and 
                                    #    only appended new songs to song_list, ignoring those already in url_dict
#                                     #driver.close()
#                                     # clean up and append song to list
#                                     my_json[i].pop('section', None)
#                                     my_json[i]['url'] = url
#                                     #f.write('\n'+str(my_json[i])+',')
#                                     song_list.append(my_json[i])

                                # this is where the above block really belongs 
                                #     so song_list gets every song, even if the url is in url_dict
                                #driver.close()
                                # clean up and append song to list
                                my_json[i].pop('section', None)
                                my_json[i]['url'] = url
                                #f.write('\n'+str(my_json[i])+',')
                                #print(my_json[i])
                                song_list.append(my_json[i])

                            if songs == 20:
                                #keepGoing = True
                                page += 1
                                myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                # slow down...
                                time.sleep(1.1)
                                if str(myData)=="<Response [429]>":
                                    #print("too many requests")
                                    time.sleep(30)
                                    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                                    if str(myData)=="<Response [429]>":
                                        time.sleep(30)
                                        print("too many requests")
                            else:
                                # update histogram of how many songs per chord progression
                                index = page*20 - 20 + songs
                                print('histo1', index)
                                histo[index] += 1
                                keepGoing = False
                        else:
                            # update histogram of how many songs per chord progression
                            index = page*20 - 20
                            print("histo2", index)
                            histo[index] += 1
                            keepGoing = False

                    except:
                        # for some reason, page=20 doesn't work on 1-5-6-4... there are other data corruptions too
                        print('2. Unable to extract for', chord1, chord2, chord3, chord4, "page", page)
                        f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
                        f2.close()
                        f2 = open(filename3se, 'a')

                        page += 1
                        myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                        # slow down...
                        time.sleep(1.1)
                        if str(myData)=="<Response [429]>":
                            #print("too many requests")
                            time.sleep(30)
                            myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
                            if str(myData)=="<Response [429]>":
                                time.sleep(30)
                                print("too many requests")
                else:
                    print("No songs match this chord progression", mykey)
                    keepGoing = False

        except:
            print('1. Unable to extract for', chord1, chord2, chord3, chord4)
            f2.write('\nUnable to extract for '+chord1+','+chord2+','+chord3+','+chord4)
            f2.close()
            f2 = open(filename3se, 'a')

            # slow down
            time.sleep(1.1)
        
        big_dict[mykey] = song_list
    #f.close()
    #f = open(filename3s, 'a')
    #time.sleep(1.1)
    j += 1
    # restart the driver every 100 chord progressions
    if j%100==0:
        driver = webdriver.Chrome(r"C:\Users\Jesse\Desktop\PSDS4900\chromedriver.exe")
    
f2.close()


with open(filename3s, "w") as f:
    json.dump(big_dict, f)

tend = time.time()
print(tend)
#f.close()
print(tend-tstart)

print(histo)
with open(histo_file, "w") as f:  
    f.write(str(histo))

with open(url_dict_file, "w") as f:
    json.dump(url_dict, f)

1615086500.0353622
500
1/500 5,2,4,67
histo1 3
2/500 3,L7,3,4/3
histo1 1
3/500 7/6,565/5,57,67
histo1 1
4/500 56/6,56/5,5/6,564/6
histo1 1
5/500 77,b7,46,M464
histo1 1
6/500 17,1,4,5
histo1 1
7/500 16,4,7/5,164
histo1 2
8/500 242,b7,46,b46
histo1 1
9/500 1,66,7,5/6
histo1 1
10/500 3,57/2,242,543
histo1 1
11/500 36,26,4,76
histo1 1
12/500 4,27,67,665
histo1 1
13/500 6,3,6,26
histo1 1
14/500 6,5,564,5
histo1 1
15/500 5,4,542,37
histo1 1
16/500 1,17,164,1
histo1 1
17/500 67,4,1,5
histo1 26
18/500 164,642,2,264
histo1 1
19/500 1,2,6,1
histo1 18
20/500 5,57/6,2,57/6
histo1 1
21/500 164,56/5,4,6
histo1 1
22/500 47,5,27,6
histo1 1
23/500 56/6,6,565/5,5
histo1 1
24/500 1,6,L26,47
histo1 1
25/500 5,37,564,6
histo1 2
26/500 56,3,364,3
histo1 1
27/500 5,b7,565,1
histo1 1
28/500 5,1,27,564
histo1 1
29/500 67,343,M565,1
histo1 2
30/500 642,47,56/5,5
histo1 1
31/500 164,4,564,6
histo1 1
32/500 16,4,b7,46
histo1 1
33/500 4,7/6,67,4
histo1 1
34/500 66,6,2,6
histo1 1
35/500 5,D3,D4,D3
histo1 1
36/500 6

histo1 1
282/500 56,3,2,D47
histo1 1
283/500 1,57/6,4,L4
histo1 1
284/500 2,5,b6,D3
histo1 1
285/500 1,6,57/5,4
histo1 1
286/500 5,26,5,1
histo1 2
287/500 1,26,6,1
histo1 1
288/500 4,b3,b7,b6
histo1 2
289/500 5,3,b1,4
histo1 1
290/500 5/5,2,b4,5
histo1 1
291/500 27,5,1,2
histo1 3
292/500 47,37,27,464
histo1 1
293/500 6,5,1,364
too many requests
2. Unable to extract for 6 5 1 364 page 1
histo2 20
294/500 37,67,b57,57/4
histo1 1
295/500 6,642,26,6
histo1 1
296/500 4,1,5,5/6
histo1 14
297/500 4,16,C16,57
histo1 1
298/500 5/5,5/6,6,b7
histo1 1
299/500 2,b7,1,56
histo1 1
300/500 6,2,643,642
histo1 1
301/500 57,1,M142,4
histo1 2
302/500 46,1,3,542
histo1 1
303/500 4,5/6,6,L27
histo1 2
304/500 27,6,27,67
histo1 1
305/500 5/6,6,5/6,27
histo1 3
306/500 2,57,2,26
histo1 1
307/500 27,1,4,5
histo1 1
308/500 264,67,27,264
histo1 1
309/500 47,1,47,1
histo1 4
310/500 4,47,67,66
histo1 1
311/500 4,464,5,56
histo1 3
312/500 3,47,b67,57
histo1 1
313/500 47,27,L77,57/6
histo1 1
314/500 6,1,5,56/6
histo1 

In [ ]:
##########################
# list errors here
##########################
73/500 3,5/2,27,57/5
too many requests
2. Unable to extract for 3 5/2 27 57/5 page 1
histo2 20

176/500 27,67,665,565/2
too many requests
2. Unable to extract for 27 67 665 565/2 page 1
histo2 20

218/500 4,1,5,6
2. Unable to extract for 4 1 5 6 page 20
histo1 476
b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'

233/500 564,57/5,5/5,564
too many requests
2. Unable to extract for 564 57/5 5/5 564 page 1
histo2 20

263/500 6,56,66,Y2
too many requests
2. Unable to extract for 6 56 66 Y2 page 1
histo2 20

293/500 6,5,1,364
too many requests
2. Unable to extract for 6 5 1 364 page 1
histo2 20

448/500 26,7/6,6,36
too many requests
2. Unable to extract for 26 7/6 6 36 page 1
histo2 20

492/500 b3,3,36,56/6
too many requests
2. Unable to extract for b3 3 36 56/6 page 1
histo2 20

In [8]:
##########################
# test errors here
##########################
mykey = '4,1,5,6'
page = 20
if page == 1:
    myData = requests.get(api_url+'trends/songs?cp='+mykey, headers=header)
else:
    myData = requests.get(api_url+'trends/songs?cp='+mykey+'&page='+str(page), headers=header)
print(myData.content)
#print(len(myData.json()))
#print(myData.json())

b'{"name":"Internal Server Error","message":"An internal server error occurred.","code":0,"status":500}'


In [9]:
# ##########################
# # fix histo here
# ##########################
# histo[20] -= 7
# print(histo)
# with open(histo_file, "w") as f:  
#     f.write(str(histo))

[0, 369, 55, 31, 10, 9, 4, 2, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0